In [48]:
import numpy as np
import pandas as pd
import scipy.integrate as integrate
from scipy.special import comb
from scipy.stats import stats

H0 = 'Кількість померлих та тих,  хто лишився живим - однакова'
H1 = 'Кількість померлих менша, аніж тих,  хто вижив'
H2 = 'Кількість померлих більша, аніж тих,  хто вижив'
H3 = 'Кількість померлих та тих,  хто лишився живим - неоднакова'

data = pd.read_csv('./src/lab1_data1.csv', sep=';')['Survival'].to_frame()

total_count = int(data.count())
survival_count = int(data[data == 1].count())
survival_p = np.float(survival_count) / np.float(data.count())
p0 = 0.5

alpha = 0.1

print(f'Обсяг вибірки становить {total_count} осіб, з них тих, хто вижив - {survival_count}\n\n')

# Загальні фунації
def select_h(h_dict, default):
    for k, v in binominal_h.items():
        if v < alpha:
            return k
    return H0




# Біномінальне розподілення 
def b(n, i, p):
    return comb(n, i) * np.float_power(p, i) * np.float_power(1 - p, n - i)

binominal_h = {
    H1: np.sum([b(total_count, i, p0) for i in range(survival_count, total_count + 1)]),
    H2: np.sum([b(total_count, i, p0) for i in range(0, survival_count + 1)])
}
binominal_h[H3] = 2 * min(binominal_h[H1], binominal_h[H2])

binominal_result = select_h(binominal_h, H0)
print(f'Використовуючи біноміальне розподілення справдилась гіпотиза `{binominal_result}`\n\n')


# розподілення відповідно до Z-критерію 
def z(p, p0, n):
    return (p - p0) / (((p0 * (1 - p0)) / n) ** 0.5)

z0 = z(survival_p, p0, total_count)
prefix = 1 / ((2 * np.pi) ** 0.5)
normal_h = {
    H1: prefix * integrate.quad(lambda x: np.exp(-(x * x / 2)), z0, np.PINF)[0],
    H2: prefix * integrate.quad(lambda x: np.exp(-(x * x / 2)), np.NINF, z0)[0],
    H3: 2 * prefix * integrate.quad(lambda x: np.exp(-(x * x / 2)), np.abs(z0), np.PINF)[0],
}
z_result = select_h(normal_h, H0)
print(f'Використовуючи Z-критерій  справдилась гіпотиза `{binominal_result}`\n\n')

# розподілення відповідно до крітерію X2
x2 = stats.chisquare([survival_count, total_count - survival_count], [total_count / 2, total_count / 2])[1]

print(f'Використовуючи Хі-квадрта  справдилась гіпотиза `{H0 if x2 >= (1 - alpha) else H3}`\n\n')


Обсяг вибірки становить 226 осіб, з них тих, хто вижив - 140


Використовуючи біноміальне розподілення справдилась гіпотиза `Кількість померлих менша, аніж тих,  хто вижив`


Використовуючи Z-критерій  справдилась гіпотиза `Кількість померлих менша, аніж тих,  хто вижив`


Використовуючи Хі-квадрта  справдилась гіпотиза `Кількість померлих та тих,  хто лишився живим - неоднакова`


